In [58]:
import os
import math
import sys
import statsmodels.api as sm
import numpy as np
from functools import reduce
from operator import and_
import yfinance as yf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import datetime as dt
from pdb import set_trace as keyboard
import xlrd
from xlrd import open_workbook
from sklearn.decomposition import FastICA
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
from tkinter import StringVar
import matplotlib.pyplot as plt
from sklearn import linear_model
clf = linear_model.LinearRegression()

import matplotlib

yf.pdr_override()

In [59]:
nDays = 45
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=nDays)

In [60]:
fund_data_raw = pd.read_excel('assets/12239939000192.xlsx')

In [61]:
mask = (fund_data_raw['Data'] > startDate) & (fund_data_raw['Data'] <= endDate)
fund_data = fund_data_raw[mask]
fund_data.set_index('Data', inplace=True)
fund_pctChange = pd.DataFrame(fund_data['Variação'])

In [62]:
ibov_data = pd.read_csv('assets/ibov.csv', sep=';')
tickers = ibov_data.iloc[:,0].values[:-2]

In [63]:
tickers_yf = [ticker + '.SA' for ticker in tickers]

startDate_str = startDate.strftime("%Y-%m-%d")
endDate_str   = endDate.strftime("%Y-%m-%d")
stocks_df_raw = pdr.get_data_yahoo(tickers_yf, start=startDate_str, end=endDate_str)['Close']

[***********           22%                       ]  15 of 67 downloaded

Exception in thread Thread-152:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_on

[******************    37%                       ]  25 of 67 downloaded

Exception in thread Thread-167:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_on

[**********************55%*                      ]  37 of 67 downloaded

Exception in thread Thread-181:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_on

[**********************75%***********            ]  50 of 67 downloaded

Exception in thread Thread-194:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_on

[*********************100%***********************]  61 of 67 downloaded


In [64]:
stocks_df = stocks_df_raw.pct_change()
stocks_df.columns = [i.replace('.SA', '') for i in stocks_df.columns]
stocks_df.head()

,ABEV3,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BRAP4,BRFS3,BRKM5,BRML3,...,SMLS3,SUZB3,TAEE11,TIMP3,UGPA3,USIM5,VALE3,VIVT4,VVAR11,WEGE3
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-17,-0.006077,0.007073,0.000193,0.003559,-0.005284,0.012579,0.000601,0.006418,-0.000835,0.012380,...,-0.019030,0.032076,NaN,-0.011254,0.004953,-0.003191,-0.006786,-0.002586,NaN,0.000888
2019-07-18,0.014452,-0.006297,0.016995,0.018322,0.021780,-0.006211,-0.006907,0.000911,0.005571,-0.005435,...,-0.016210,-0.007770,NaN,0.004065,0.026614,0.001067,-0.001898,-0.005741,NaN,0.009765
2019-07-19,-0.008767,-0.015842,-0.012533,-0.012478,-0.022615,-0.026786,0.001210,0.014867,-0.004155,-0.012978,...,-0.012696,-0.040603,NaN,-0.012146,-0.030725,0.000000,0.002472,0.001118,NaN,0.001758
2019-07-22,-0.005528,0.024022,0.000192,0.010579,0.013830,0.012844,-0.005436,0.032885,-0.018359,-0.010381,...,-0.007934,-0.011790,NaN,-0.003279,-0.041605,-0.020256,-0.005121,-0.002791,NaN,-0.008337


In [65]:
stocks_df.columns

Index(['ABEV3', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BRAP4', 'BRFS3',
       'BRKM5', 'BRML3', 'BTOW3', 'CCRO3', 'CIEL3', 'CMIG4', 'CPFE3', 'CPLE6',
       'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6',
       'EMBR3', 'ENBR3', 'EQTL3', 'ESTC3', 'FIBR3', 'FLRY3', 'GGBR4', 'GOAU4',
       'GOLL4', 'HYPE3', 'IGTA3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'KROT3',
       'LAME4', 'LREN3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NATU3', 'PCAR4',
       'PETR3', 'PETR4', 'QUAL3', 'RADL3', 'RAIL3', 'RENT3', 'SANB11',
       'SAPR11', 'SBSP3', 'SMLS3', 'SUZB3', 'TAEE11', 'TIMP3', 'UGPA3',
       'USIM5', 'VALE3', 'VIVT4', 'VVAR11', 'WEGE3'],
      dtype='object')

In [90]:
data = fund_pctChange.join(stocks_df)
data.fillna(data.mean(), inplace=True)
columns_with_problem = data.columns.to_series()[np.isnan(data).any()]
data.drop(columns_with_problem, axis=1, inplace=True)

In [91]:
y = data.iloc[:,0]
X_total = data.iloc[:, 1:]

In [92]:
class Results:
    def __init__(self, rsquared_adj, pvalues, selected_stocks):
        self.rsquared_adj = rsquared_adj
        self.pvalues = pvalues
        self.selected_stocks = selected_stocks

In [93]:
#columns_with_problem = X_total.columns.to_series()[np.isnan(X_total).any()]
#X_total.drop(columns_with_problem, axis=1, inplace=True)

In [94]:
nCols = X_total.columns.size
nCols

61

In [95]:
predef_stocks = []
stocks_to_sample = [x for x in X_total.columns if not x in predef_stocks]

from pdb import set_trace as keyboard

N = 300
sensitivity = .15
size_samples = np.random.randint(12-len(predef_stocks), 20-len(predef_stocks), size=N)

result_list = []
rsquared_ajd_list = []

X_total_model = X_total.copy()
X_total_model['Bias'] = 1.

useBias = 0

for iteration in range(N):
    
    sys.stdout.write("\rIteration number {}".format(iteration))
    sys.stdout.flush()
    
    while True:
        
        selected_stocks = np.random.choice(stocks_to_sample, size=size_samples[iteration]).tolist() + \
                                                                    predef_stocks
        if useBias:
             selected_stocks += ['Bias']

        X = X_total_model[selected_stocks]

        regressor_OLS = sm.OLS(endog=y, exog=X.values).fit()  
        pvalues_dict = dict(zip(selected_stocks, regressor_OLS.pvalues))

        try:
        
            while max(list(pvalues_dict.values())) > sensitivity:
                stocks_bigPvalue = list({ key:value for (key,value) in pvalues_dict.items() if value > sensitivity}.keys())
                if 'Bias' in stocks_bigPvalue:
                    stocks_bigPvalue.remove('Bias')
                stockToRemove = np.random.choice(stocks_bigPvalue)
                selected_stocks.remove(stockToRemove)
                stocks_bigPvalue.remove(stockToRemove)
                regressor_OLS = sm.OLS(endog=y, exog=X.values).fit()  
                pvalues_dict = dict(zip(selected_stocks, regressor_OLS.pvalues))

            result_list.append(Results(regressor_OLS.rsquared_adj, 
                               pvalues_dict, 
                               selected_stocks))
        
        except:
            pass
        
        if len(pvalues_dict) > 0:
            break
                          


    
from operator import attrgetter
best_rsquaredAdj_obj = max(result_list, key=attrgetter('rsquared_adj'))

print('Number of stocks: %s' % len(best_rsquaredAdj_obj.selected_stocks))
print('\nStocks: %s' % best_rsquaredAdj_obj.selected_stocks)
print('\nR2 adj: %s' % best_rsquaredAdj_obj.rsquared_adj)
print('\np-values: %s' % pd.DataFrame(best_rsquaredAdj_obj.pvalues.values(), list(best_rsquaredAdj_obj.pvalues.keys())))

Iteration number 299Number of stocks: 1

Stocks: ['EGIE3']

R2 adj: 0.9669174492371031

p-values:               0
EGIE3  0.002306


ENBR3  0.012173
MULT3  0.008534
HYPE3  0.010997
EMBR3  0.018648
ELET6  0.007146
JBSS3  0.016430
BRAP4  0.018931
SUZB3  0.011507
CSNA3  0.006397
WEGE3  0.015269
CIEL3  0.005272
ITSA4  0.004171
BTOW3  0.014274
NATU3  0.005441




In [ ]:
print('\np-values: %s' % best_rsquaredAdj_obj.pvalues)

In [ ]:
pd.DataFrame(best_rsquaredAdj_obj.pvalues.values(), list(best_rsquaredAdj_obj.pvalues.keys()))

In [ ]:
list(best_rsquaredAdj_obj.pvalues.keys())

In [ ]:
selected_stocks.remove('CCRO3')